# Advanced Features of stackelberg-opt

This notebook explores advanced features including:
- Custom evaluation metrics
- Semantic constraint extraction
- Population management strategies
- Checkpointing and recovery
- Visualization tools
- Performance optimization

In [ ]:
# Import all required components
from stackelberg_opt import *
from stackelberg_opt.components import *
from stackelberg_opt.utils import *

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import asyncio
from typing import Dict, List, Tuple, Optional

# Set up visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Advanced features loaded!")

## 1. Semantic Constraint Extraction

Extract and analyze constraints from module prompts and execution traces:

In [ ]:
# Create a system with explicit constraints
constrained_modules = {
    "validator": Module(
        name="validator",
        prompt="""You MUST validate all inputs before processing.
ALWAYS check for: length limits, format requirements, and security issues.
You should prefer structured output formats.
Never process inputs longer than 1000 characters.
Ensure all outputs are JSON-compatible.""",
        module_type=ModuleType.LEADER,
        dependencies=[]
    ),
    "processor": Module(
        name="processor",
        prompt="""Process the validated input according to these rules:
- Must maintain data integrity
- Should optimize for speed when possible
- Always log processing steps""",
        module_type=ModuleType.FOLLOWER,
        dependencies=["validator"]
    )
}

# Extract constraints
constraint_extractor = SemanticConstraintExtractor()
leader_constraints = constraint_extractor.extract_constraints(
    "validator",
    constrained_modules["validator"],
    {}  # No traces yet
)

print("Extracted Constraints for Leader Module:")
print("=" * 50)
print(f"\nHard constraints ({len(leader_constraints['hard'])}):") 
for constraint in leader_constraints['hard']:
    print(f"  - {constraint}")

print(f"\nSoft constraints ({len(leader_constraints['soft'])}):") 
for constraint in leader_constraints['soft']:
    print(f"  - {constraint}")

print(f"\nImplicit constraints ({len(leader_constraints['implicit'])}):") 
for constraint in leader_constraints['implicit']:
    print(f"  - {constraint}")

## 2. Advanced Population Management

Explore different selection strategies and archive management:

In [ ]:
# Create a population manager with custom settings
pop_manager = PopulationManager(
    max_size=20,
    elite_size=5,
    diversity_weight=0.3,
    innovation_weight=0.2,
    selection_pressure=2.0
)

# Create diverse candidates
print("Creating diverse population...")
for i in range(15):
    # Create candidate with varying scores
    candidate = SystemCandidate(
        modules=constrained_modules.copy(),
        candidate_id=i,
        generation=0
    )
    
    # Simulate different performance profiles
    if i < 5:  # High performers
        candidate.scores = {j: 0.8 + np.random.rand() * 0.2 for j in range(3)}
    elif i < 10:  # Medium performers
        candidate.scores = {j: 0.5 + np.random.rand() * 0.3 for j in range(3)}
    else:  # Diverse/innovative candidates
        candidate.scores = {j: 0.3 + np.random.rand() * 0.4 for j in range(3)}
    
    candidate.equilibrium_value = np.random.rand()
    candidate.stability_score = np.random.rand()
    
    added, reason = pop_manager.add_candidate(candidate, i // 5)
    if added:
        print(f"  Added candidate {i}: {reason}")

# Display population statistics
stats = pop_manager.get_statistics()
print(f"\nPopulation Statistics:")
print(f"  Total population: {stats['population_size']}")
print(f"  Elite archive: {stats['elite_size']}")
print(f"  Diversity archive: {stats['diversity_size']}")
print(f"  Innovation archive: {stats['innovation_size']}")

In [ ]:
# Test different parent selection strategies
selection_methods = ['tournament', 'roulette', 'diverse', 'elite']
selection_results = {}

print("Testing Parent Selection Strategies:")
print("=" * 50)

for method in selection_methods:
    parents = pop_manager.select_parents(n=5, method=method)
    avg_fitness = np.mean([p.get_average_score() for p in parents])
    
    selection_results[method] = {
        'parents': parents,
        'avg_fitness': avg_fitness,
        'ids': [p.candidate_id for p in parents]
    }
    
    print(f"\n{method.capitalize()} Selection:")
    print(f"  Selected IDs: {selection_results[method]['ids']}")
    print(f"  Average fitness: {avg_fitness:.3f}")

# Visualize selection bias
fig, ax = plt.subplots(figsize=(10, 6))

methods = list(selection_results.keys())
avg_scores = [selection_results[m]['avg_fitness'] for m in methods]

bars = ax.bar(methods, avg_scores, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
ax.set_ylabel('Average Fitness of Selected Parents')
ax.set_title('Parent Selection Strategy Comparison')
ax.set_ylim(0, 1)

# Add value labels on bars
for bar, score in zip(bars, avg_scores):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{score:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 3. Custom Evaluation Metrics

Implement custom evaluation metrics for domain-specific optimization:

In [ ]:
class CustomEvaluator:
    """Custom evaluator with domain-specific metrics."""
    
    def __init__(self, base_evaluator):
        self.base_evaluator = base_evaluator
        self.custom_metrics = {}
    
    def add_metric(self, name: str, weight: float, metric_fn):
        """Add a custom metric to the evaluation."""
        self.custom_metrics[name] = {
            'weight': weight,
            'function': metric_fn
        }
    
    async def evaluate_with_custom_metrics(self, candidate, input_data, expected_output):
        """Evaluate candidate with base + custom metrics."""
        # Get base evaluation
        output, trace = await self.base_evaluator(candidate.modules, input_data)
        
        # Calculate base score
        base_score = self.base_evaluator._calculate_score(output, expected_output, trace)
        
        # Calculate custom metrics
        custom_scores = {}
        for metric_name, metric_info in self.custom_metrics.items():
            score = metric_info['function'](output, trace, candidate)
            custom_scores[metric_name] = score * metric_info['weight']
        
        # Combine scores
        total_weight = 1.0 + sum(m['weight'] for m in self.custom_metrics.values())
        final_score = (base_score + sum(custom_scores.values())) / total_weight
        
        # Store detailed metrics in trace
        trace.custom_metrics = custom_scores
        trace.final_score = final_score
        
        return output, trace

# Define custom metrics
def response_time_metric(output, trace, candidate):
    """Penalize slow responses."""
    total_time = sum(trace.module_timings.values())
    # Score decreases with time (max 1.0 for instant, 0.0 for >1 second)
    return max(0, 1 - total_time)

def consistency_metric(output, trace, candidate):
    """Reward consistent module scores."""
    if trace.intermediate_scores:
        scores = list(trace.intermediate_scores.values())
        return 1 - np.std(scores)  # Lower variance = higher score
    return 0.5

def innovation_metric(output, trace, candidate):
    """Reward novel outputs."""
    # Simple proxy: length variation from average
    avg_length = 100  # Assumed average
    length_diff = abs(len(str(output)) - avg_length) / avg_length
    return min(1.0, length_diff * 2)  # Cap at 1.0

# Create mock base evaluator
async def mock_base_evaluator(modules, input_data):
    trace = ExecutionTrace()
    trace.module_timings = {'mod1': 0.1, 'mod2': 0.2}
    trace.intermediate_scores = {'mod1': 0.8, 'mod2': 0.7}
    trace.success = True
    return "Mock output", trace

mock_base_evaluator._calculate_score = lambda o, e, t: 0.75

# Set up custom evaluator
custom_eval = CustomEvaluator(mock_base_evaluator)
custom_eval.add_metric('response_time', 0.2, response_time_metric)
custom_eval.add_metric('consistency', 0.15, consistency_metric)
custom_eval.add_metric('innovation', 0.1, innovation_metric)

print("Custom evaluator configured with metrics:")
for name, info in custom_eval.custom_metrics.items():
    print(f"  - {name}: weight={info['weight']}")

## 4. Checkpointing and Recovery

Demonstrate saving and loading optimization state:

In [ ]:
# Create checkpoint manager
checkpoint_dir = Path("optimization_checkpoints")
checkpoint_manager = CheckpointManager(checkpoint_dir=checkpoint_dir)

# Create auto-checkpointer
auto_checkpointer = AutoCheckpointer(
    checkpoint_manager,
    time_interval=300,  # Every 5 minutes
    iteration_interval=10  # Every 10 iterations
)

# Simulate optimization state
optimization_state = {
    'generation': 15,
    'evaluations_used': 150,
    'best_candidate': pop_manager.get_best_candidate(),
    'population': pop_manager.population[:5],  # Save subset
    'generation_stats': {
        14: {'avg_fitness': 0.65, 'best_fitness': 0.82},
        15: {'avg_fitness': 0.68, 'best_fitness': 0.85}
    },
    'config': {
        'budget': 1000,
        'population_size': 20,
        'mutation_rate': 0.7
    }
}

# Save checkpoint
checkpoint_name = "optimization_example"
checkpoint_manager.save_checkpoint(optimization_state, checkpoint_name)
print(f"Checkpoint saved: {checkpoint_name}")

# List available checkpoints
checkpoints = checkpoint_manager.list_checkpoints()
print(f"\nAvailable checkpoints:")
for cp in checkpoints:
    print(f"  - {cp['name']} (created: {cp['created']})")
    if 'summary' in cp:
        print(f"    Generation: {cp['summary'].get('generation', 'N/A')}")

In [ ]:
# Load checkpoint and verify
loaded_state = checkpoint_manager.load_checkpoint(checkpoint_name)

print("Loaded checkpoint contents:")
print(f"  Generation: {loaded_state['generation']}")
print(f"  Evaluations used: {loaded_state['evaluations_used']}")
print(f"  Population size: {len(loaded_state['population'])}")
print(f"  Config: {json.dumps(loaded_state['config'], indent=2)}")

# Verify best candidate
if loaded_state['best_candidate']:
    best = loaded_state['best_candidate']
    print(f"\nBest candidate from checkpoint:")
    print(f"  ID: {best.candidate_id}")
    print(f"  Average score: {best.get_average_score():.3f}")

## 5. Advanced Visualization

Create comprehensive visualizations of the optimization process:

In [ ]:
# Generate synthetic optimization history for visualization
def generate_synthetic_history(generations=20):
    """Generate synthetic optimization history."""
    history = {}
    
    for gen in range(generations):
        # Simulate improving performance
        base_fitness = 0.4 + (gen / generations) * 0.4
        history[gen] = {
            'avg_fitness': base_fitness + np.random.normal(0, 0.05),
            'best_fitness': base_fitness + 0.1 + np.random.normal(0, 0.02),
            'diversity': 0.5 * np.exp(-gen/10) + 0.2,  # Decreasing diversity
            'innovations': max(0, int(5 * np.exp(-gen/5) + np.random.normal(0, 1))),
            'equilibrium': 0.5 + (gen / generations) * 0.3 + np.random.normal(0, 0.03),
            'stability': 0.6 + (gen / generations) * 0.2 + np.random.normal(0, 0.02)
        }
    
    return history

# Create synthetic history
synthetic_history = generate_synthetic_history(25)

# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Comprehensive Optimization Analysis', fontsize=16)

generations = list(synthetic_history.keys())

# 1. Fitness progression
ax1 = axes[0, 0]
avg_fitness = [synthetic_history[g]['avg_fitness'] for g in generations]
best_fitness = [synthetic_history[g]['best_fitness'] for g in generations]
ax1.plot(generations, avg_fitness, 'b-', label='Average', linewidth=2)
ax1.plot(generations, best_fitness, 'r-', label='Best', linewidth=2)
ax1.fill_between(generations, avg_fitness, best_fitness, alpha=0.3)
ax1.set_title('Fitness Evolution')
ax1.set_xlabel('Generation')
ax1.set_ylabel('Fitness Score')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Diversity over time
ax2 = axes[0, 1]
diversity = [synthetic_history[g]['diversity'] for g in generations]
ax2.plot(generations, diversity, 'g-', linewidth=2)
ax2.set_title('Population Diversity')
ax2.set_xlabel('Generation')
ax2.set_ylabel('Diversity Score')
ax2.grid(True, alpha=0.3)

# 3. Innovation frequency
ax3 = axes[0, 2]
innovations = [synthetic_history[g]['innovations'] for g in generations]
ax3.bar(generations, innovations, color='orange', alpha=0.7)
ax3.set_title('Innovation Frequency')
ax3.set_xlabel('Generation')
ax3.set_ylabel('Number of Innovations')
ax3.grid(True, alpha=0.3, axis='y')

# 4. Equilibrium vs Performance
ax4 = axes[1, 0]
equilibrium = [synthetic_history[g]['equilibrium'] for g in generations]
ax4.scatter(best_fitness, equilibrium, c=generations, cmap='viridis', s=50)
ax4.set_title('Equilibrium vs Performance')
ax4.set_xlabel('Best Fitness')
ax4.set_ylabel('Equilibrium Value')
ax4.grid(True, alpha=0.3)
# Add colorbar
cbar = plt.colorbar(ax4.collections[0], ax=ax4)
cbar.set_label('Generation')

# 5. Stability progression
ax5 = axes[1, 1]
stability = [synthetic_history[g]['stability'] for g in generations]
ax5.plot(generations, stability, 'm-', linewidth=2)
ax5.fill_between(generations, stability, alpha=0.3, color='magenta')
ax5.set_title('System Stability')
ax5.set_xlabel('Generation')
ax5.set_ylabel('Stability Score')
ax5.grid(True, alpha=0.3)

# 6. Multi-objective radar chart
ax6 = axes[1, 2]
# Get final generation metrics
final_gen = max(generations)
final_metrics = synthetic_history[final_gen]

categories = ['Fitness', 'Equilibrium', 'Stability', 'Diversity']
values = [
    final_metrics['best_fitness'],
    final_metrics['equilibrium'],
    final_metrics['stability'],
    final_metrics['diversity']
]

# Create radar chart
angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
values += values[:1]  # Complete the circle
angles += angles[:1]

ax6 = plt.subplot(2, 3, 6, projection='polar')
ax6.plot(angles, values, 'o-', linewidth=2, color='red')
ax6.fill(angles, values, alpha=0.25, color='red')
ax6.set_theta_offset(np.pi / 2)
ax6.set_theta_direction(-1)
ax6.set_xticks(angles[:-1])
ax6.set_xticklabels(categories)
ax6.set_ylim(0, 1)
ax6.set_title('Final Multi-Objective Profile')
ax6.grid(True)

plt.tight_layout()
plt.show()

## 6. Performance Optimization with Caching

Demonstrate the caching system for improved performance:

In [ ]:
# Create caching system
llm_cache = LLMCache(cache_dir=Path("llm_cache"))
computation_cache = ComputationCache(max_size=1000)

# Simulate LLM calls with caching
print("Testing LLM Cache:")
print("=" * 50)

prompts = [
    "Optimize this prompt for clarity",
    "Optimize this prompt for clarity",  # Duplicate
    "Generate a summary of the text",
    "Optimize this prompt for clarity"  # Another duplicate
]

for i, prompt in enumerate(prompts):
    # Check cache first
    cached_response = llm_cache.get(prompt, "gpt-3.5-turbo", 0.7)
    
    if cached_response:
        print(f"\nCall {i+1}: CACHE HIT")
        print(f"  Prompt: {prompt}")
        print(f"  Response: {cached_response[:50]}...")
    else:
        print(f"\nCall {i+1}: CACHE MISS")
        print(f"  Prompt: {prompt}")
        # Simulate LLM response
        response = f"Optimized version of '{prompt}'"
        llm_cache.set(prompt, "gpt-3.5-turbo", 0.7, response)
        print(f"  Response: {response}")

# Display cache statistics
cache_stats = llm_cache.get_stats()
print(f"\nCache Statistics:")
print(f"  Total entries: {cache_stats['total_entries']}")
print(f"  Cache size: {cache_stats['cache_size_bytes']} bytes")
print(f"  Hit rate: {2/4:.1%} (2 hits out of 4 calls)")

In [ ]:
# Test computation cache with complex operations
print("\nTesting Computation Cache:")
print("=" * 50)

def expensive_computation(x, y, operation='multiply'):
    """Simulate expensive computation."""
    import time
    time.sleep(0.1)  # Simulate delay
    
    if operation == 'multiply':
        return x * y
    elif operation == 'power':
        return x ** y
    else:
        return x + y

# Test caching
test_cases = [
    (10, 20, 'multiply'),
    (10, 20, 'multiply'),  # Cache hit
    (5, 3, 'power'),
    (10, 20, 'multiply'),  # Another cache hit
    (5, 3, 'power')  # Cache hit
]

import time

for x, y, op in test_cases:
    # Create cache key
    cache_key = computation_cache.make_key('expensive_computation', x=x, y=y, operation=op)
    
    # Check cache
    cached_result = computation_cache.get(cache_key)
    
    if cached_result is not None:
        print(f"\nCACHE HIT: {op}({x}, {y}) = {cached_result}")
        print("  Time saved: ~0.1 seconds")
    else:
        print(f"\nCACHE MISS: Computing {op}({x}, {y})...")
        start_time = time.time()
        result = expensive_computation(x, y, op)
        computation_time = time.time() - start_time
        
        # Store in cache
        computation_cache.set(cache_key, result)
        
        print(f"  Result: {result}")
        print(f"  Computation time: {computation_time:.3f} seconds")

# Display computation cache stats
comp_stats = computation_cache.get_stats()
print(f"\nComputation Cache Statistics:")
print(f"  Total entries: {comp_stats['total_entries']}")
print(f"  Total accesses: {comp_stats['total_accesses']}")
print(f"  Cache efficiency: {3/5:.1%} (3 hits out of 5 calls)")

## 7. Integration Example: Complete Advanced System

Combine all advanced features in a complete example:

In [ ]:
class AdvancedOptimizationSystem:
    """Complete system using all advanced features."""
    
    def __init__(self, modules: Dict[str, Module]):
        self.modules = modules
        
        # Initialize components
        self.constraint_extractor = SemanticConstraintExtractor()
        self.dependency_analyzer = DependencyAnalyzer()
        self.population_manager = PopulationManager(
            max_size=30,
            elite_size=8,
            diversity_weight=0.35
        )
        
        # Caching
        self.llm_cache = LLMCache()
        self.computation_cache = ComputationCache()
        
        # Checkpointing
        self.checkpoint_manager = CheckpointManager()
        self.auto_checkpointer = AutoCheckpointer(
            self.checkpoint_manager,
            iteration_interval=5
        )
        
        # Visualization
        self.visualizer = OptimizationVisualizer()
        
        # Extract initial constraints
        self.constraints = self._extract_all_constraints()
        
    def _extract_all_constraints(self):
        """Extract constraints from all modules."""
        constraints = {}
        for name, module in self.modules.items():
            constraints[name] = self.constraint_extractor.extract_constraints(
                name, module, {}
            )
        return constraints
    
    def analyze_system(self):
        """Comprehensive system analysis."""
        print("System Analysis Report")
        print("=" * 60)
        
        # Dependency analysis
        dep_analysis = self.dependency_analyzer.analyze_dependencies(self.modules)
        print(f"\n1. Dependency Structure:")
        print(f"   - Is DAG: {dep_analysis['properties']['is_dag']}")
        print(f"   - Max depth: {dep_analysis['properties']['max_depth']}")
        print(f"   - Topological order: {dep_analysis['properties']['topological_order']}")
        
        # Constraint summary
        print(f"\n2. Constraint Summary:")
        for module_name, module_constraints in self.constraints.items():
            total_constraints = (
                len(module_constraints['hard']) +
                len(module_constraints['soft']) +
                len(module_constraints['implicit'])
            )
            print(f"   - {module_name}: {total_constraints} total constraints")
        
        # Module types
        print(f"\n3. Module Distribution:")
        type_counts = {}
        for module in self.modules.values():
            type_name = module.module_type.value
            type_counts[type_name] = type_counts.get(type_name, 0) + 1
        
        for type_name, count in type_counts.items():
            print(f"   - {type_name}: {count} modules")
    
    def create_dashboard(self):
        """Create a comprehensive dashboard."""
        fig = plt.figure(figsize=(16, 10))
        fig.suptitle('Advanced Optimization System Dashboard', fontsize=20)
        
        # Grid layout
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
        
        # Module type distribution (pie chart)
        ax1 = fig.add_subplot(gs[0, 0])
        type_counts = {}
        for module in self.modules.values():
            type_name = module.module_type.value
            type_counts[type_name] = type_counts.get(type_name, 0) + 1
        
        ax1.pie(type_counts.values(), labels=type_counts.keys(), autopct='%1.0f%%')
        ax1.set_title('Module Type Distribution')
        
        # Constraint complexity (bar chart)
        ax2 = fig.add_subplot(gs[0, 1])
        module_names = list(self.constraints.keys())
        constraint_counts = [
            len(self.constraints[m]['hard']) + 
            len(self.constraints[m]['soft'])
            for m in module_names
        ]
        
        ax2.bar(module_names, constraint_counts, color='skyblue')
        ax2.set_title('Constraint Complexity by Module')
        ax2.set_xlabel('Module')
        ax2.set_ylabel('Number of Constraints')
        ax2.tick_params(axis='x', rotation=45)
        
        # Population statistics (if available)
        ax3 = fig.add_subplot(gs[0, 2])
        pop_stats = self.population_manager.get_statistics()
        stats_data = [
            pop_stats['population_size'],
            pop_stats['elite_size'],
            pop_stats['diversity_size'],
            pop_stats['innovation_size']
        ]
        stats_labels = ['Population', 'Elite', 'Diversity', 'Innovation']
        
        ax3.bar(stats_labels, stats_data, color=['blue', 'gold', 'green', 'red'])
        ax3.set_title('Population Archives')
        ax3.set_ylabel('Size')
        
        # Cache performance
        ax4 = fig.add_subplot(gs[1, :])
        cache_data = {
            'LLM Cache Entries': self.llm_cache.get_stats()['total_entries'],
            'Computation Cache Entries': self.computation_cache.get_stats()['total_entries'],
            'Total Cache Accesses': self.computation_cache.get_stats()['total_accesses']
        }
        
        ax4.barh(list(cache_data.keys()), list(cache_data.values()), color='purple')
        ax4.set_title('Caching System Performance')
        ax4.set_xlabel('Count')
        
        # System info text
        ax5 = fig.add_subplot(gs[2, :])
        ax5.axis('off')
        
        info_text = f"""System Configuration:
        • Total Modules: {len(self.modules)}
        • Population Size: {self.population_manager.max_size}
        • Elite Archive Size: {self.population_manager.elite_size}
        • Checkpoint Interval: {self.auto_checkpointer.iteration_interval} iterations
        • Cache Hit Rate: Varies by usage pattern
        """
        
        ax5.text(0.1, 0.5, info_text, fontsize=12, verticalalignment='center',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        plt.tight_layout()
        return fig

# Create advanced system
advanced_modules = {
    "orchestrator": Module(
        name="orchestrator",
        prompt="Orchestrate the entire workflow. You MUST coordinate all modules.",
        module_type=ModuleType.LEADER,
        dependencies=[]
    ),
    "analyzer": Module(
        name="analyzer",
        prompt="Analyze inputs based on orchestrator guidance.",
        module_type=ModuleType.FOLLOWER,
        dependencies=["orchestrator"]
    ),
    "optimizer": Module(
        name="optimizer",
        prompt="Optimize the analyzed results. Ensure maximum efficiency.",
        module_type=ModuleType.FOLLOWER,
        dependencies=["analyzer"]
    ),
    "validator": Module(
        name="validator",
        prompt="Validate all outputs. Must check for errors and consistency.",
        module_type=ModuleType.INDEPENDENT,
        dependencies=["optimizer"]
    )
}

# Initialize and analyze
advanced_system = AdvancedOptimizationSystem(advanced_modules)
advanced_system.analyze_system()

# Create dashboard
dashboard = advanced_system.create_dashboard()
plt.show()

## Summary

This notebook demonstrated advanced features of stackelberg-opt:

### Key Advanced Features:

1. **Semantic Constraint Extraction**
   - Automatically extract hard/soft constraints from prompts
   - Analyze implicit requirements and dependencies

2. **Advanced Population Management**
   - Multiple selection strategies (tournament, roulette, diverse, elite)
   - Separate archives for elite, diversity, and innovation
   - Configurable selection pressure

3. **Custom Evaluation Metrics**
   - Add domain-specific metrics
   - Weighted combination of multiple objectives
   - Fine-grained performance analysis

4. **Checkpointing and Recovery**
   - Save/load optimization state
   - Automatic checkpointing at intervals
   - Resume interrupted optimizations

5. **Advanced Visualization**
   - Multi-faceted performance analysis
   - Evolution tracking over generations
   - Population diversity visualization
   - Multi-objective radar charts

6. **Performance Optimization**
   - LLM response caching
   - Computation result caching
   - Significant speedup for repeated operations

### Best Practices:

- **Use caching** for expensive LLM calls and computations
- **Enable checkpointing** for long-running optimizations
- **Monitor population diversity** to avoid premature convergence
- **Extract and respect constraints** for valid optimizations
- **Visualize progress** to understand optimization dynamics

For production use, combine these features to create robust, efficient optimization systems tailored to your specific domain.